In [1]:
import pandas as pd
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://openairal.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = open("Azurekey.txt","r").read()

#strprompt="Red or green what colour is grass? Grass is "
strprompt="Microsoft Azure is great because"

response = openai.Completion.create(
  engine="textdavinci003",
  prompt=strprompt, 
  temperature=0.1,
  max_tokens=4000,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  logprobs=10
)

response

#Show the probabilites of the next words in this sentence - green should be set to  " green": -0.23977444, whic is 78% probability


<OpenAIObject text_completion id=cmpl-6x1offEofXhrphebYqHthANJZxZU6 at 0x23561dd90e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": {
        "text_offset": [
          32,
          35,
          44,
          46,
          51,
          57,
          60,
          69,
          73,
          82,
          87,
          91,
          94,
          99,
          102,
          108,
          109,
          116,
          117,
          121,
          128,
          141,
          145,
          154,
          157,
          161,
          167,
          168,
          171,
          178,
          180,
          185,
          191,
          194,
          203,
          204,
          214,
          222,
          231,
          232,
          240,
          241,
          251,
          252,
          263,
          264,
          274,
          275,
          279,
          284,
          285,
          288,
          293,
          

In [2]:
[n for n in response.to_dict_recursive().items() if n[0] == "choices"][0][1][0]["text"]

' it provides a wide range of services and features that can be used to build, deploy, and manage applications and services in the cloud. It offers a wide range of services, including virtual machines, storage, databases, networking, analytics, and more. It also provides a secure and reliable platform for businesses to build and deploy applications. Additionally, Azure provides a wide range of tools and services to help businesses manage their cloud infrastructure and applications.'

In [3]:
pd.DataFrame([n for n in response.to_dict_recursive().items() if n[0] == "choices"][0][1][0]["logprobs"])

,tokens,token_logprobs,top_logprobs,text_offset
0,it,-0.064436,"{' ': -6.7658625, ' Azure': -8.059223, ' Micro...",32
1,provides,-1.260314,"{' allows': -2.864723, ' can': -4.7739563, ' e...",35
2,a,-0.709287,"{' a': -0.70928663, ' an': -2.337714, ' busine...",44
3,wide,-1.181915,"{' cloud': -2.6748805, ' complete': -4.234281,...",46
4,range,-0.303856,"{' and': -8.484248, ' array': -2.8024554, ' as...",51
...,...,...,...,...
83,infrastructure,-1.360646,"{' applications': -2.8940928, ' deployments': ...",484
84,and,-1.165194,"{' and': -1.1651936, ' cost': -6.2683415, ' ea...",499
85,applications,-1.190417,"{' applications': -1.1904168, ' data': -4.3535...",503
86,.,-0.307744,"{' and': -5.659061, ' at': -6.5428557, ' easil...",516


In [4]:
#Get the percentage chance of the next word

import math
import numpy

dffull = pd.DataFrame()

count=0
while (count < 10):
    try:
        df = pd.DataFrame([n for n in response.to_dict_recursive().items() if n[0] == "choices"][0][1][0]["logprobs"]["top_logprobs"][count].items())
        df = df.sort_values(by=[1], ascending=False)
        df.reset_index(drop=True, inplace=True)
        dffull[count*2-1] = df[0]
        dffull[count*2] = numpy.exp(df[1])*100
        count=count+1
    except:
        print("out of bounds")
        count=count+1

#df = pd.DataFrame([n for n in response.to_dict_recursive().items() if n[0] == "choices"][0][1][0]["logprobs"]["top_logprobs"][2].items())
#df = df.sort_values(by=[1], ascending=False)
#df.reset_index(drop=True, inplace=True)
#dffull[2] = df[0]
#dffull[3] = numpy.exp(df[1])*100

dffull

,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,it,93.759587,is,30.329092,a,49.199505,wide,30.669089,range,73.796742,of,99.967979,services,51.555329,and,28.695711,features,41.222845,that,44.707268
1,of,5.163928,provides,28.356497,an,9.654810,comprehensive,21.072850,variety,19.290321,of,0.025183,cloud,36.763363,that,23.055674,tools,26.664196,to,18.682566
2,its,0.399177,offers,25.967218,businesses,9.091973,cloud,6.891506,array,6.066093,and,0.001638,features,2.702770,",",12.945954,solutions,11.003861,.,11.149648
3,\n,0.115245,allows,5.699892,users,8.521803,variety,5.540476,selection,0.584555,\n,0.001293,tools,2.334855,for,12.581392,products,5.012808,for,10.428793
4,Microsoft,0.097794,'s,1.958913,cloud,6.030122,reliable,5.045085,-,0.058545,or,0.000578,products,1.454267,to,12.555215,capabilities,4.538552,",",8.070108
5,you,0.094665,has,1.703440,organizations,2.430245,range,5.014203,set,0.048976,,0.000547,powerful,0.847447,.,2.020385,resources,2.547607,designed,1.235235
6,:,0.058657,enables,1.605023,developers,1.947720,secure,3.335017,breadth,0.027828,services,0.000432,options,0.571659,from,1.582025,technologies,1.267080,which,0.902341
7,there,0.047771,can,0.844690,many,1.263819,powerful,2.128792,assortment,0.022312,cloud,0.000269,solutions,0.451263,in,1.102591,options,1.025921,such,0.762595
8,the,0.037500,makes,0.838246,reliable,0.985340,lot,1.795785,and,0.020670,for,0.000152,computing,0.281092,including,0.916417,benefits,0.935456,in,0.599787
9,Azure,0.031617,gives,0.561835,the,0.965172,complete,1.449022,suite,0.017543,a,0.000145,different,0.264226,such,0.733062,is,0.900329,including,0.512037
